In [36]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import re

In [12]:
df = pd.read_csv('../Input/attacks.csv', encoding_errors='ignore')

In [15]:
#Delete useless cols
columns_to_delete = ["Case Number", "pdf", "href formula", "href", "Case Number.1", "Case Number.2", "original order", "Unnamed: 22", "Unnamed: 23"]
df = df.drop(columns_to_delete, axis=1)

In [21]:
#Check rows filled with NaN's and delete them
df = df[~df.isna().all(1)] 

In [23]:
df

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
0,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF"
1,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonMcNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com"
2,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com"
3,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF"
4,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234"
6298,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234"
6299,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF"
6300,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8N, 79W",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938"


In [31]:
#Check Nans in columns (we don't discard any column)
df.isna().sum().sort_values()

Date                         0
Year                         2
Type                         4
Investigator or Source      17
Injury                      28
Country                     50
Name                       210
Area                       455
Fatal (Y/N)                539
Location                   540
Activity                   544
Sex                        565
Age                       2831
Species                   2838
Time                      3354
dtype: int64

In [34]:
# Clean 'Date' col
df.Date.unique()

array(['25-Jun-2018', '18-Jun-2018', '09-Jun-2018', ..., '1900-1905',
       '1883-1889', '1845-1853'], dtype=object)

In [44]:
# get rid of "Reported", "before" ...

# S[S.str.match(r'(^P.*)')==True]

df.Date.str.contains(r"-[A-Za-z]{3}-").value_counts()

True     5392
False     910
Name: Date, dtype: int64

In [49]:
df[df.Date.str.contains(r"[A-Za-z]{3}-")]

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
0,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF"
1,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonMcNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com"
2,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com"
3,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF"
4,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6246,Before 24 Apr-1959,0.0,Unprovoked,BERMUDA,Paget,Paget Parish,Spearfishing,Ross Doe,M,NaN,Shoulder abraded by skin of shark,N,NaN,NaN,Mentioned in letter from L. S. Mowbray dated 4...
6257,Before Mar-1956,0.0,Unprovoked,NORTH PACIFIC OCEAN,NaN,Wake Island,"Fishing, wading with string of fish",male,M,NaN,Survived,N,NaN,NaN,"J. Oetzel, Skin Diver Magazine, March 1956, p.19"
6288,Before 17-Jul-1916,0.0,Unprovoked,USA,North Carolina,Somewhere between Hatteras and Beaufort,Swimming,"""youthful male""",M,NaN,"""Lost leg""",N,NaN,NaN,"C. Creswell, GSAF; Wilmington Star, 7/17/1916"
6289,No date (3 days after preceding incident) & pr...,0.0,Unprovoked,SOUTH AFRICA,KwaZulu-Natal,Durban,Fishing,a native fisherman,M,NaN,"FATAL, body not recovered but shark was caught...",Y,NaN,NaN,"Rural New Yorker, 7/19/1913"


In [ ]:
#re.split(pattern, string, maxsplit=0, flags=0)
pattern = "[A-Za-z]{3}-"
re.split(pattern, df.Date)

In [67]:
df['Country'].str.isupper().value_counts()

True    6252
Name: Country, dtype: int64

In [66]:
#We change all elements in 'Country' into uppercase
df['Country'] = df['Country'].str.upper()

/var/folders/_3/sh9lq8cx353bw8jn9pybzhn00000gn/T/ipykernel_7593/912665611.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Country = df['Country'].str.upper()


In [76]:
df.Activity.value_counts()[:10]

Surfing         971
Swimming        869
Fishing         431
Spearfishing    333
Bathing         162
Wading          149
Diving          127
Standing         99
Snorkeling       89
Scuba diving     76
Name: Activity, dtype: int64

In [77]:
df["Month"] = df.Date.apply(lambda x: "".join(re.findall(r"[A-Za-z]{3}-",x)).replace("-",""))

/var/folders/_3/sh9lq8cx353bw8jn9pybzhn00000gn/T/ipykernel_7593/3547653603.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Month"] = df.Date.apply(lambda x: "".join(re.findall(r"[A-Za-z]{3}-",x)).replace("-",""))
